In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
import time

chrome_options = Options()
chrome_options.set_capability("goog:loggingPrefs", {"performance": "ALL"})
driver = webdriver.Chrome(options=chrome_options)

driver.get("https://rahavard365.com/asset/453/posts")

# Scroll function (same as before)
scroll_script = """
    const scroller = document.querySelector("div.PostItem_root__vCOCA");
    scroller.scrollTop += 500;
"""
driver.execute_script(scroll_script)
time.sleep(2)  # Wait for potential requests

# Get network logs
logs = driver.get_log("performance")

# Save for analysis
with open("network_logs.json", "w") as f:
    json.dump(logs, f)

# Filter for API calls
api_calls = [entry for entry in logs if "Network.requestWillBeSent" in entry["message"]]
print(f"Found {len(api_calls)} potential API calls during scrolling")

Found 377 potential API calls during scrolling


In [7]:
import json
from urllib.parse import urlparse

def analyze_api_calls(logs_file):
    with open(logs_file) as f:
        logs = json.load(f)
    
    api_endpoints = {}
    
    for entry in logs:
        try:
            message = json.loads(entry['message'])
            if message.get('message', {}).get('method') == 'Network.responseReceived':
                url = message['message']['params']['response']['url']
                content_type = message['message']['params']['response']['headers'].get('content-type', '')
                
                if 'application/json' in content_type:
                    parsed = urlparse(url)
                    # Group by path without query parameters
                    base_path = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"
                    
                    if base_path not in api_endpoints:
                        api_endpoints[base_path] = {
                            'count': 0,
                            'example_url': url,
                            'methods': set(),
                            'sizes': []
                        }
                    
                    api_endpoints[base_path]['count'] += 1
                    api_endpoints[base_path]['methods'].add(
                        message['message']['params']['request']['method']
                    )
                    api_endpoints[base_path]['sizes'].append(
                        message['message']['params']['response']['headers'].get('content-length', 0)
                    )
        except:
            continue
    
    # Sort by most frequently called endpoints
    sorted_apis = sorted(api_endpoints.items(), key=lambda x: x[1]['count'], reverse=True)
    
    print("Most likely data APIs:")
    for i, (path, data) in enumerate(sorted_apis[:10], 1):  # Top 10 candidates
        avg_size = sum(map(int, data['sizes'])) / max(1, len(data['sizes']))
        print(f"{i}. {path}")
        print(f"   Calls: {data['count']} | Methods: {', '.join(data['methods'])}")
        print(f"   Avg size: {avg_size:.0f} bytes | Example: {data['example_url']}")
        print()

analyze_api_calls('network_logs.json')

Most likely data APIs:
1. https://rahavard365.com/v2/statics/loaders/SplashScreen.json
   Calls: 4 | Methods: 
   Avg size: 0 bytes | Example: https://rahavard365.com/v2/statics/loaders/SplashScreen.json

2. https://rahavard365.com/api/v2/licenses
   Calls: 2 | Methods: 
   Avg size: 0 bytes | Example: https://rahavard365.com/api/v2/licenses

3. https://rahavard365.com/api/v2/social/posts
   Calls: 2 | Methods: 
   Avg size: 0 bytes | Example: https://rahavard365.com/api/v2/social/posts?exclude_replies=true&has_chart_analysis=false&_count=16&entity_type=exchange.asset

4. https://rahavard365.com/api/v2/ip-check
   Calls: 1 | Methods: 
   Avg size: 0 bytes | Example: https://rahavard365.com/api/v2/ip-check

5. https://rahavard365.com/api/v2/asset/453/header
   Calls: 1 | Methods: 
   Avg size: 0 bytes | Example: https://rahavard365.com/api/v2/asset/453/header

6. https://rahavard365.com/api/v2/page-settings
   Calls: 1 | Methods: 
   Avg size: 0 bytes | Example: https://rahavard365.com/

In [25]:
import requests
import time
import json
from datetime import datetime

# Configuration - Edit these as needed
API_URL = "https://rahavard365.com/api/v2/social/posts"
OUTPUT_FILE = f"rahavard_posts_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
MAX_PAGES = 10  # Set to None to scrape all available pages
DELAY_BETWEEN_REQUESTS = 1.5  # Seconds

# Setup headers to mimic browser request
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
#     "Accept": "application/json",
#     "Referer": "https://rahavard365.com/social"
# }


headers = {
    "authority": "rahavard365.com",
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9",
    "application-name": "rahavard",
    "content-type": "application/json",
    "referer": "https://rahavard365.com/social",
    "sec-ch-ua": '"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}

# Variables to track progress
all_posts = []
current_page = 1
has_more = True
retry_count = 0
max_retries = 3

print(f"Starting to scrape posts from {API_URL}")

while has_more and retry_count < max_retries:
    if MAX_PAGES and current_page > MAX_PAGES:
        break
    
    # Prepare request parameters
    params = {
        "exclude_replies": "true",
        "has_chart_analysis": "false",
        "entity_type": "exchange.asset",
        "_entity_id": 453,
        "_count": 16,  # Max items per request
        "_page": current_page,
        "_t": int(time.time() * 1000),  # Cache buster
        
    }
    
    try:
        # Make the API request
        print(f"Requesting page {current_page}...")
        response = requests.get(
            API_URL,
            params=params,
            headers=headers,
            timeout=10
        )
        
        # Check if request was successful
        if response.status_code == 200:
            data = response.json()
            
            # Process the response
            if isinstance(data, dict) and data.get("data"):
                new_posts = data["data"]
                
                if not new_posts:
                    print("No more posts found")
                    has_more = False
                else:
                    all_posts.extend(new_posts)
                    print(f"Page {current_page}: Got {len(new_posts)} posts (Total: {len(all_posts)})")
                    current_page += 1
                    retry_count = 0  # Reset retry counter
            else:
                print("Unexpected response format - retrying...")
                retry_count += 1
        else:
            print(f"HTTP {response.status_code} - Retrying...")
            retry_count += 1
            
        # Pause between requests
        time.sleep(DELAY_BETWEEN_REQUESTS)
        
    except Exception as e:
        print(f"Error: {str(e)} - Retrying...")
        retry_count += 1
        time.sleep(5)

# Save the results
print(f"\nFinished scraping. Total posts collected: {len(all_posts)}")
print(f"Saving to {OUTPUT_FILE}")

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_posts, f, ensure_ascii=False, indent=2)

print("Done!")

Starting to scrape posts from https://rahavard365.com/api/v2/social/posts
Requesting page 1...
Page 1: Got 16 posts (Total: 16)
Requesting page 2...
Page 2: Got 16 posts (Total: 32)
Requesting page 3...
Page 3: Got 16 posts (Total: 48)
Requesting page 4...
Page 4: Got 16 posts (Total: 64)
Requesting page 5...
Page 5: Got 16 posts (Total: 80)
Requesting page 6...
Page 6: Got 16 posts (Total: 96)
Requesting page 7...
Page 7: Got 16 posts (Total: 112)
Requesting page 8...
Page 8: Got 16 posts (Total: 128)
Requesting page 9...
Page 9: Got 16 posts (Total: 144)
Requesting page 10...
Page 10: Got 16 posts (Total: 160)

Finished scraping. Total posts collected: 160
Saving to rahavard_posts_20250630_022841.json
Done!


In [11]:
import os
print(os.getcwd())

C:\Users\Partiran


In [12]:
all_posts[1]

{'id': '16684044',
 'body': 'بچه\u200cها فکرکنم دوباره قراره جنگ بشه',
 'creation_date_time': '2025-06-30T01:37:51+03:30',
 'entity_id': '1560',
 'entity_type': 'exchange.asset',
 'entity_info': {'id': '1560',
  'type': 'exchange.asset',
  'info': {'english_trade_symbol': '',
   'title': 'طلا گرم 18 عیار',
   'english_title': 'Gold 18K',
   'short_title': 'طلا گرم 18 عیار',
   'english_short_name': 'Gold 18K',
   'slug': 'طلا_گرم_18_عیار'}},
 'account': {'id': '1877341',
  'username': 'Mohsen.rmz',
  'avatar_id': '',
  'full_name': 'محسن رمز'},
 'replies_count': 0,
 'upvote_count': 1,
 'downvote_count': 1,
 'state_id': '1',
 'state': 'updated'}

In [28]:
import requests
import time
import json
from datetime import datetime

# Configuration
API_URL = "https://rahavard365.com/api/v2/social/posts"
OUTPUT_FILE = f"filtered_posts_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
TARGET_ENTITY_ID = "453"  # specific entity ID
MAX_REQUESTS = 500
DELAY = 1.5  # seconds

# Headers from browser inspection
headers = {
    "authority": "rahavard365.com",
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9",
    "application-name": "rahavard",
    "content-type": "application/json",
    "referer": "https://rahavard365.com/social",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}

# Initialize storage
filtered_posts = []
last_id = None
request_count = 0

print(f"Starting to collect posts for entity ID: {TARGET_ENTITY_ID}")

while request_count < MAX_REQUESTS:
    # Prepare parameters
    params = {
        "exclude_replies": "true",
        "has_chart_analysis": "false",
        "_count": 16,
        "entity_id": TARGET_ENTITY_ID,
        "entity_type": "exchange.asset",
        "_t": int(time.time() * 1000),
    }
    
    if last_id:
        params["before_id"] = last_id
    
    try:
        print(f"\nRequest #{request_count + 1}")
        # print(f"Params: {params}")
        
        response = requests.get(
            API_URL,
            params=params,
            headers=headers,
            timeout=10
        )
        request_count += 1
        
        print(f"Status: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, dict) and data.get("data"):
                batch = data["data"]
                
                if not batch:
                    print("No more posts available")
                    break
                
                # Verify entity_id in each post
                valid_posts = [
                    p for p in batch 
                    if str(p.get("entity_id")) == TARGET_ENTITY_ID
                ]
                
                if len(valid_posts) != len(batch):
                    print(f"Filtered {len(batch) - len(valid_posts)} posts with wrong entity_id")
                
                if valid_posts:
                    filtered_posts.extend(valid_posts)
                    last_id = valid_posts[-1]["id"]
                    # print(f"Added {len(valid_posts)} valid posts (Total: {len(filtered_posts)})")
                else:
                    print("No valid posts in this batch")
                    break
            else:
                print("Unexpected response format")
                break
        else:
            print(f"HTTP Error {response.status_code}")
            break
            
        time.sleep(DELAY)
        
    except Exception as e:
        print(f"Error: {str(e)}")
        break

# Save results
print(f"\nFinished. Collected {len(filtered_posts)} posts for entity {TARGET_ENTITY_ID}")
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(filtered_posts, f, ensure_ascii=False, indent=2)

print(f"Results saved to {OUTPUT_FILE}")

# Verification
if filtered_posts:
    print("\nVerification:")
    print(f"First post entity_id: {filtered_posts[0].get('entity_id')}")
    print(f"Last post entity_id: {filtered_posts[-1].get('entity_id')}")
    print(f"Unique entity_ids found: {len(set(p.get('entity_id') for p in filtered_posts))}")

Starting to collect posts for entity ID: 453

Request #1
Status: 200

Request #2
Status: 200

Request #3
Status: 200

Request #4
Status: 200

Request #5
Status: 200

Request #6
Status: 200

Request #7
Status: 200

Request #8
Status: 200

Request #9
Status: 200

Request #10
Status: 200

Request #11
Status: 200

Request #12
Status: 200

Request #13
Status: 200

Request #14
Status: 200

Request #15
Status: 200

Request #16
Status: 200

Request #17
Status: 200

Request #18
Status: 200

Request #19
Status: 200

Request #20
Status: 200

Request #21
Status: 200

Request #22
Status: 200

Request #23
Status: 200

Request #24
Status: 200

Request #25
Status: 200

Request #26
Status: 200

Request #27
Status: 200

Request #28
Status: 200

Request #29
Status: 200

Request #30
Status: 200

Request #31
Status: 200

Request #32
Status: 200

Request #33
Status: 200

Request #34
Status: 200

Request #35
Status: 200

Request #36
Status: 200

Request #37
Status: 200

Request #38
Status: 200

Request #39
S

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd


driver = webdriver.Chrome()
driver.get("https://rahavard365.com/asset/453/trades")
soup = BeautifulSoup(driver.page_source, 'html.parser')

table = soup.select_one('div.ant-table-content > table')

if table is None:
    raise ValueError("Table not found. Check if page requires JavaScript rendering.")

# Step 3: Extract headers
headers = []
for th in table.select('thead th.ant-table-cell'):
    header_text = th.get_text(strip=True)
    headers.append(header_text)

# Step 4: Extract rows with proper handling of number formatting
data = []
for row in table.select('tbody tr.ant-table-row'):
    row_data = []
    for td in row.select('td.ant-table-cell'):
        # Handle the numeric cells with NumberHandler classes
        num_handler = td.select_one('.NumberHandler_price__ew5nF')
        if num_handler:
            value = num_handler.get_text(strip=True)
            # Handle special cases like "—"
            if 'MutedValue_muted__eLxQJ' in str(td):
                value = ''
        else:
            value = td.get_text(strip=True)
        row_data.append(value)
    data.append(row_data)

# Step 5: Create DataFrame and save to CSV
df = pd.DataFrame(data, columns=headers)
df.to_csv('extracted_table.csv', index=False, encoding='utf-8-sig')

print("Table extracted successfully!")
print(f"Found {len(df)} rows with columns: {list(df.columns)}")

ValueError: Table not found. Check if page requires JavaScript rendering.

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configure Selenium
service = Service('path/to/chromedriver')  # Download from https://chromedriver.chromium.org/
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in background
options.add_argument('--disable-gpu')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

driver = webdriver.Chrome(service=service, options=options)

try:
    # Load the page
    driver.get("https://rahavard365.com/asset/453/trades")
    
    # Wait for table to load (adjust timeout as needed)
    wait = WebDriverWait(driver, 15)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.ant-table-content table')))
    
    # Optional: Scroll to ensure all data loads
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Allow time for loading
    
    # Get the page source and parse
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract the table
    table = soup.select_one('div.ant-table-content > table')
    
    if table:
        # Extract headers
        headers = [th.get_text(strip=True) for th in table.select('thead th.ant-table-cell')]
        
        # Extract rows
        data = []
        for row in table.select('tbody tr.ant-table-row'):
            row_data = []
            for td in row.select('td.ant-table-cell'):
                num_handler = td.select_one('.NumberHandler_price__ew5nF')
                if num_handler:
                    value = num_handler.get_text(strip=True)
                    if 'MutedValue_muted__eLxQJ' in str(td):
                        value = ''
                else:
                    value = td.get_text(strip=True)
                row_data.append(value)
            data.append(row_data)
        
        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        df.to_csv('extracted_data.csv', index=False, encoding='utf-8-sig')
        print(f"Success! Saved {len(df)} rows to extracted_data.csv")
    else:
        print("Table found in DOM but couldn't parse structure. Saving HTML for debugging...")
        with open('debug_page.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)

finally:
    driver.quit()

NoSuchDriverException: Message: Unable to locate or obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_table_from_rahavard(url):
    """
    Extracts table data from the specified Rahavard365 URL.

    Args:
        url (str): The URL of the Rahavard365 page containing the table.

    Returns:
        pandas.DataFrame: A DataFrame containing the extracted table data, or None if extraction fails.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table by its class or structure.
    # Based on the HTML you provided, the table is inside a div with class "ant-table-content"
    table_container = soup.find('div', class_='ant-table-content')

    if not table_container:
        print("Table container not found. Check the HTML structure or class name.")
        return None

    table = table_container.find('table')

    if not table:
        print("Table not found inside the container.")
        return None

    # Extract table headers
    headers = []
    for th in table.find('thead').find_all('th'):
        # Extract text from div within th, and strip whitespace
        header_text = th.find('div').get_text(strip=True) if th.find('div') else th.get_text(strip=True)
        headers.append(header_text)

    # Extract table rows
    table_data = []
    for row in table.find('tbody').find_all('tr', class_='ant-table-row'):
        row_data = []
        for td in row.find_all('td', class_='ant-table-cell'):
            # Extract text from div within td, if it exists, otherwise get text directly from td
            cell_text = td.find('div').get_text(strip=True) if td.find('div') else td.get_text(strip=True)
            row_data.append(cell_text)
        table_data.append(row_data)

    # Create a Pandas DataFrame
    try:
        df = pd.DataFrame(table_data, columns=headers)
        return df
    except ValueError as e:
        print(f"Error creating DataFrame: {e}")
        print("Headers:", headers)
        print("First few rows of data:", table_data[:2]) # Print first 2 rows for debugging
        return None

# URL of the page you want to extract the table from
url = "https://rahavard365.com/asset/453/trades"

# Call the function to extract the table
df = extract_table_from_rahavard(url)

if df is not None:
    print("Table extracted successfully:")
    print(df.head()) # Print the first few rows of the DataFrame
    # You can now save the DataFrame to a CSV, Excel, or process it further
    # df.to_csv("rahavard_trades.csv", index=False, encoding='utf-8-sig')
else:
    print("Failed to extract table data.")

Error fetching the URL: 403 Client Error: Forbidden for url: https://rahavard365.com/asset/453/trades
Failed to extract table data.


In [6]:
html = '<table style="table-layout: auto;"><colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col style="width: 1000px;"></colgroup><thead class="ant-table-thead"><tr><th class="ant-table-cell"><div style="white-space: nowrap;">تاریخ</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">باز</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">بیشترین</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">کمترین</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">پایانی</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">تغییر قیمت پایانی</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">درصد تغییر قیمت پایانی</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">آخرین</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">تغییر آخرین قیمت</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">درصد تغییر آخرین قیمت</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">حجم</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">ارزش</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">دفعات</div></th><th class="ant-table-cell"><div style="white-space: nowrap;">تعداد خریدار</div></th></tr></thead><tbody class="ant-table-tbody"><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/04/08</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬293</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬293</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬293</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬293</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-101</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫98٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬293</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-101</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫98٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۵۵٬۳۴۳٬۹۶۸"><div class="NumberHandler_price__ew5nF NumberHandler__price">55٫344</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱۸۲٬۲۴۷٬۶۸۶٬۶۲۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">182٫25</div><span title="میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">B</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٬156</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">755</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/04/07</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬394</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬394</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬394</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬394</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-104</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫97٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬394</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-104</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫97٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۵۱٬۸۹۱٬۲۶۶"><div class="NumberHandler_price__ew5nF NumberHandler__price">51٫891</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱۷۶٬۱۱۸٬۹۵۶٬۸۰۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">176٫12</div><span title="میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">B</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">774</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">442</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/21</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬530</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬538</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬483</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬498</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">14</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫4٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬500</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">16</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫46٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۲۲۳٬۸۱۹٬۹۷۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">223٫82</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۷۸۲٬۹۸۷٬۳۵۸٬۳۷۸"><div class="NumberHandler_price__ew5nF NumberHandler__price">782٫99</div><span title="میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">B</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">7٬826</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٬489</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/20</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬480</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬540</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬450</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬484</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">3</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫09٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬514</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">33</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫95٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۲۹۹٬۱۵۰٬۳۷۸"><div class="NumberHandler_price__ew5nF NumberHandler__price">299٫15</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۰۴۲٬۱۵۴٬۶۳۵٬۶۹۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫04</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">9٬805</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">3٬600</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/19</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬575</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬610</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬469</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬481</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-95</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫66٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬471</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-105</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫94٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۶۴۰٬۱۰۳٬۱۹۶"><div class="NumberHandler_price__ew5nF NumberHandler__price">640٫103</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۲۲۸٬۰۴۲٬۵۲۶٬۱۱۶"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫23</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">17٬285</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price"><span class="MutedValue_muted__eLxQJ">—</span></div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/18</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬540</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬654</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬533</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬576</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">4</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫11٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬560</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-12</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫34٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۳۲۶٬۲۴۸٬۷۶۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">326٫249</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۱۶۶٬۵۵۱٬۴۴۹٬۳۹۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫17</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">11٬716</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">4٬028</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/17</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬623</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬656</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬561</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬572</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-99</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫7٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬561</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-110</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-3٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۵۴۷٬۲۸۵٬۵۲۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">547٫286</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۹۵۵٬۱۳۸٬۵۹۹٬۱۰۱"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫96</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">13٬087</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬018</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/13</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬739</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬750</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬641</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬671</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-81</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫16٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬643</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-109</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫91٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۴۴۲٬۸۹۸٬۲۱۹"><div class="NumberHandler_price__ew5nF NumberHandler__price">442٫898</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۶۲۵٬۷۲۵٬۴۳۳٬۰۴۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫63</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">16٬624</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬674</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/12</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬759</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬815</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬728</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬752</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-6</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫16٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬730</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-28</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫75٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۲۸۵٬۲۵۳٬۵۴۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">285٫254</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۰۷۰٬۲۷۷٬۵۴۹٬۳۴۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫07</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">11٬696</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">4٬450</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/11</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬829</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬829</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬728</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬758</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-81</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫11٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬764</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-75</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-1٫95٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۳۷۹٬۶۳۸٬۱۸۷"><div class="NumberHandler_price__ew5nF NumberHandler__price">379٫638</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۴۲۶٬۶۴۷٬۰۶۳٬۲۶۵"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫43</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">12٬537</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price"><span class="MutedValue_muted__eLxQJ">—</span></div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/10</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬900</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬902</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬786</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬839</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">6</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫16٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬793</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-40</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-1٫04٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۳۷۶٬۶۳۶٬۴۱۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">376٫636</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۴۴۵٬۸۴۱٬۰۸۸٬۰۸۷"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫45</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">12٬036</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price"><span class="MutedValue_muted__eLxQJ">—</span></div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/07</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬870</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬873</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬812</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬833</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-41</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-1٫06٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬830</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-44</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-1٫14٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۳۵۵٬۸۶۱٬۱۳۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">355٫861</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۳۶۳٬۸۴۷٬۳۶۸٬۳۴۲"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫36</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">11٬390</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">3٬999</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/06</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬919</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬939</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬847</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬874</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-4</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫1٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬849</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-29</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫75٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۴۲۹٬۶۱۹٬۰۵۵"><div class="NumberHandler_price__ew5nF NumberHandler__price">429٫619</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۶۶۴٬۳۸۴٬۷۰۹٬۰۳۵"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫66</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">13٬625</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬230</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/05</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬930</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬943</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬831</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬878</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">40</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">1٫04٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬904</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">66</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">1٫72٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۵۵۳٬۲۲۳٬۲۳۲"><div class="NumberHandler_price__ew5nF NumberHandler__price">553٫223</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۱۴۵٬۳۲۲٬۵۹۰٬۰۱۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫15</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">12٬926</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">4٬569</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/04</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬844</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬892</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬798</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬838</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">12</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫31٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬810</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-16</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫42٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۶۰۲٬۰۱۷٬۶۹۱"><div class="NumberHandler_price__ew5nF NumberHandler__price">602٫018</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۳۱۰٬۶۵۹٬۶۹۸٬۹۲۱"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫31</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">16٬728</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬391</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/03/03</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬770</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬844</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬685</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬826</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">93</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫49٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬844</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">111</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫97٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۸۹۸٬۲۷۹٬۹۷۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">898٫28</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۳٬۴۳۷٬۱۰۹٬۹۳۴٬۷۱۹"><div class="NumberHandler_price__ew5nF NumberHandler__price">3٫44</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">13٬518</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">4٬237</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/31</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬708</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬779</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬708</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬733</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-86</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫25٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬769</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-50</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-1٫31٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۴۸۹٬۶۸۶٬۳۰۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">489٫686</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۸۲۸٬۱۸۵٬۸۶۰٬۴۴۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫83</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">15٬817</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">6٬271</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/30</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬873</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬910</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬789</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬819</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-87</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫23٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬789</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-117</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-3٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۷۰۶٬۴۸۹٬۶۵۶"><div class="NumberHandler_price__ew5nF NumberHandler__price">706٫49</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۶۹۸٬۳۹۹٬۳۸۲٬۹۹۹"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫7</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">22٬458</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">8٬791</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/29</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬962</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬980</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬873</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬906</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-58</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-1٫46٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬880</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-84</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫12٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۳۷۷٬۷۵۲٬۵۳۷"><div class="NumberHandler_price__ew5nF NumberHandler__price">377٫753</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۴۷۵٬۵۴۱٬۹۵۶٬۷۵۲"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫48</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">15٬171</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬504</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/28</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬949</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬000</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬933</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬964</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">15</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫38٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬970</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">21</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫53٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۴۹۲٬۶۶۶٬۸۳۷"><div class="NumberHandler_price__ew5nF NumberHandler__price">492٫667</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۹۵۳٬۱۷۰٬۸۶۱٬۰۵۲"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫95</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">14٬293</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬417</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/27</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬000</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬030</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬924</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬949</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-96</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫37٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬933</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-112</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫77٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۹۸۲٬۱۲۰٬۴۷۵"><div class="NumberHandler_price__ew5nF NumberHandler__price">982٫12</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۳٬۸۷۸٬۲۸۸٬۱۷۳٬۸۴۶"><div class="NumberHandler_price__ew5nF NumberHandler__price">3٫88</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">25٬888</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">8٬682</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/24</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬055</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬093</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬984</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬045</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-10</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫25٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬070</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">15</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫37٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۶۲۴٬۹۰۷٬۶۳۸"><div class="NumberHandler_price__ew5nF NumberHandler__price">624٫908</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۵۲۷٬۹۵۰٬۱۶۴٬۵۸۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫53</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">15٬681</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬943</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/23</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬070</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬133</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬964</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬055</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">33</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫82٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬063</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">41</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">1٫02٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۶۳۷٬۴۹۵٬۳۱۸"><div class="NumberHandler_price__ew5nF NumberHandler__price">637٫495</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۵۸۵٬۱۲۸٬۳۳۷٬۹۷۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫59</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">16٬508</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬600</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/22</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬022</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬022</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬018</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬022</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">117</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">3٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬022</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">117</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">3٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۲۸۲٬۶۸۲٬۴۶۹"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫283</div><span title="میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">B</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۵٬۱۵۸٬۹۲۹٬۷۸۴٬۱۸۱"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٫16</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">13٬886</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬012</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/21</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬900</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬969</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬867</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬905</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-55</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-1٫39٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬889</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-71</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-1٫79٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۳۹۹٬۵۱۵٬۸۳۷"><div class="NumberHandler_price__ew5nF NumberHandler__price">399٫516</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۵۶۰٬۲۴۵٬۵۰۵٬۰۶۵"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫56</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">14٬154</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬588</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/20</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬965</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬035</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬959</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬960</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-121</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫96٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬959</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-122</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫99٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۴۶۰٬۶۳۸٬۵۳۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">460٫639</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۸۲۴٬۱۸۴٬۴۳۷٬۴۱۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫82</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">12٬311</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price"><span class="MutedValue_muted__eLxQJ">—</span></div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/17</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬098</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬108</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬027</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬081</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">92</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫31٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬059</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">70</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">1٫75٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۷۵۵٬۷۵۲٬۱۱۶"><div class="NumberHandler_price__ew5nF NumberHandler__price">755٫752</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۳٬۰۸۳٬۹۴۰٬۲۷۱٬۶۱۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">3٫08</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">17٬264</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">6٬077</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/16</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬035</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬085</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬930</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬989</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-30</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫75٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬018</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-1</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫02٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۵۷۱٬۵۶۹٬۳۴۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">571٫569</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۲۷۹٬۸۱۹٬۰۶۳٬۵۰۸"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫28</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">17٬138</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">6٬622</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/15</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬050</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬118</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬959</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬019</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-15</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫37٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬981</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-53</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-1٫31٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۴۵۶٬۴۳۴٬۴۰۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">456٫434</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۸۳۴٬۵۳۱٬۱۷۰٬۶۸۸"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫83</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">16٬551</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">6٬073</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/14</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬019</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬098</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬019</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬034</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-109</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫63٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬034</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-109</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫63٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۶۱۰٬۷۴۵٬۷۹۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">610٫746</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۴۶۳٬۸۸۷٬۴۵۷٬۰۱۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫46</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">20٬474</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">8٬807</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/13</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬143</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬144</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬143</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬143</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-128</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-3٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬143</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-128</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-3٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۵۰۴٬۳۵۷٬۳۵۹"><div class="NumberHandler_price__ew5nF NumberHandler__price">504٫357</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۰۸۹٬۵۵۴٬۱۱۳٬۰۹۹"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫09</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">9٬256</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">4٬241</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/10</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬250</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬313</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬224</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬271</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">17</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫4٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬277</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">23</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫54٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۵۵۹٬۲۱۸٬۲۳۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">559٫218</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۳۸۸٬۵۵۵٬۳۱۷٬۱۸۲"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫39</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">16٬006</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">6٬078</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/09</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬227</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬324</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬175</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬254</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">49</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">1٫17٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬270</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">65</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">1٫55٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۵۷۱٬۳۶۷٬۳۰۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">571٫367</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۴۳۰٬۳۵۲٬۹۳۴٬۲۸۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫43</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">14٬925</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬019</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/08</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬289</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬391</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬174</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬205</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-98</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫28٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬208</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-95</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫21٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۲۰۰٬۹۱۰٬۳۷۵"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫201</div><span title="میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">B</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۵٬۰۴۹٬۵۲۱٬۸۷۴٬۵۹۲"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٫05</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">28٬551</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">10٬377</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/07</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬302</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬330</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬302</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬303</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-132</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫98٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬302</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-133</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-3٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۸۲۹٬۵۶۸٬۷۳۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">829٫569</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۳٬۵۷۰٬۰۱۸٬۱۲۵٬۵۳۱"><div class="NumberHandler_price__ew5nF NumberHandler__price">3٫57</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">18٬190</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">8٬093</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/06</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬520</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬520</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬300</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬435</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">46</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">1٫05٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬413</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">24</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫55٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۲۲۹٬۹۷۳٬۴۹۷"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫23</div><span title="میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">B</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۵٬۴۵۵٬۵۱۷٬۸۲۸٬۶۱۹"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٫46</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">27٬072</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">9٬811</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/03</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬330</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬439</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬200</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬389</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">79</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">1٫83٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬439</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">129</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫99٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۱۴۱٬۱۷۵٬۱۱۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫141</div><span title="میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">B</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۹٬۳۹۷٬۸۶۰٬۲۸۳٬۵۰۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">9٫4</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">36٬567</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">10٬890</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/02</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬340</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬360</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬305</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬310</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-128</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫88٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬305</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-133</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-3٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۱۳۳٬۵۱۷٬۸۱۶"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫134</div><span title="میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">B</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۴٬۸۸۵٬۰۷۳٬۰۴۴٬۵۴۲"><div class="NumberHandler_price__ew5nF NumberHandler__price">4٫89</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">26٬178</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">10٬735</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/02/01</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬479</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬481</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬280</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬438</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">87</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬320</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-31</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫71٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۵۰۹٬۸۱۳٬۶۱۹"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫51</div><span title="میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">B</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱۱٬۱۳۸٬۰۴۱٬۵۸۰٬۷۱۲"><div class="NumberHandler_price__ew5nF NumberHandler__price">11٫14</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">52٬958</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price"><span class="MutedValue_muted__eLxQJ">—</span></div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/01/31</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬351</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬351</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬351</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬351</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">126</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫98٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬351</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">126</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫98٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۳۸۵٬۵۱۱٬۸۹۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫386</div><span title="میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">B</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۶٬۰۲۸٬۳۶۲٬۲۴۶٬۴۴۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">6٫03</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">13٬661</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">4٬836</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/01/30</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬170</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬253</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬150</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬225</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">94</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫28٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬240</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">109</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫64٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۶۶۳٬۹۶۴٬۷۵۹"><div class="NumberHandler_price__ew5nF NumberHandler__price">663٫965</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۸۰۵٬۰۸۳٬۸۲۰٬۷۸۵"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫81</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">16٬102</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬267</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/01/27</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬138</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬198</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬093</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬131</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-21</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫51٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬112</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-40</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫96٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۴۲۹٬۴۰۶٬۰۵۱"><div class="NumberHandler_price__ew5nF NumberHandler__price">429٫406</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۷۷۳٬۸۹۲٬۳۲۷٬۹۷۲"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫77</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">12٬310</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">4٬573</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/01/26</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬246</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬246</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬144</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬152</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-120</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫81٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬144</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-128</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-3٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۷۱۷٬۰۰۶٬۰۸۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">717٫006</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۲٬۹۷۷٬۲۳۹٬۷۷۰٬۰۲۱"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٫98</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">13٬913</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">5٬044</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/01/25</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬294</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬294</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬200</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬272</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">103</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫47٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬259</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">90</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫16٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۹۳۳٬۹۲۱٬۷۱۵"><div class="NumberHandler_price__ew5nF NumberHandler__price">933٫922</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۳٬۹۸۹٬۴۹۸٬۷۴۵٬۳۶۶"><div class="NumberHandler_price__ew5nF NumberHandler__price">3٫99</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">20٬726</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">7٬529</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/01/24</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬169</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬169</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬169</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬169</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">121</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫99٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬169</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">121</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫99٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۴۳۶٬۸۰۸٬۳۰۶"><div class="NumberHandler_price__ew5nF NumberHandler__price">436٫808</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۸۲۱٬۰۵۳٬۸۲۷٬۷۱۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫82</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">4٬864</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٬425</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/01/23</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬070</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬080</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬018</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬048</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNeutral__uedRs NumberHandler__price">0</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNeutral__uedRs NumberHandler__price">0٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬040</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-8</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫2٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۲۸۴٬۲۹۱٬۰۲۶"><div class="NumberHandler_price__ew5nF NumberHandler__price">284٫291</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۱۵۰٬۷۸۷٬۲۴۸٬۰۲۵"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫15</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">8٬342</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٬473</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/01/20</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬110</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬110</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬983</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬048</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">54</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">1٫35٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬010</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">16</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">0٫4٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۳۷۵٬۹۹۲٬۷۸۸"><div class="NumberHandler_price__ew5nF NumberHandler__price">375٫993</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۵۲۱٬۸۷۳٬۴۲۸٬۰۰۴"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫52</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">8٬318</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">3٬417</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/01/19</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬994</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬994</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬994</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬994</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">116</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫99٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬994</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">116</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_pricePositive__tVD_8 NumberHandler__price">2٫99٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۲۰۰٬۸۴۴٬۶۴۹"><div class="NumberHandler_price__ew5nF NumberHandler__price">200٫845</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۸۰۲٬۱۷۳٬۵۲۸٬۱۰۶"><div class="NumberHandler_price__ew5nF NumberHandler__price">802٫17</div><span title="میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">B</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٬983</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">181</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/01/18</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">3٬943</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬943</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬835</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬878</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-74</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-1٫87٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬920</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-32</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-0٫81٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۲۹۶٬۱۱۰٬۰۳۵"><div class="NumberHandler_price__ew5nF NumberHandler__price">296٫11</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۱۴۸٬۳۳۵٬۸۶۳٬۸۰۳"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫15</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">8٬808</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">2٬739</div></div></div></td></tr><tr class="ant-table-row ant-table-row-level-0"><td class="ant-table-cell"><div style="min-width: 60px;"><span class="">1404/01/17</span></div></td><td class="ant-table-cell"><div style="min-width: 40px;">4٬024</div></td><td class="ant-table-cell"><div style="min-width: 50px;">4٬024</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬914</div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬952</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-83</div></div></div></td><td class="ant-table-cell"><div style="min-width: 120px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫06٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;">3٬942</div></td><td class="ant-table-cell"><div style="min-width: 90px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-93</div></div></div></td><td class="ant-table-cell"><div style="min-width: 125px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler_priceNegative__Lxr_l NumberHandler__price">-2٫3٪</div></div></div></td><td class="ant-table-cell"><div style="min-width: 40px;"><div class="NumberHandler_numberHandler__oHINz" title="۲۹۸٬۴۶۳٬۷۸۱"><div class="NumberHandler_price__ew5nF NumberHandler__price">298٫464</div><span title="میلیون" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">M</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz" title="۱٬۱۷۹٬۶۰۲٬۴۴۷٬۳۱۰"><div class="NumberHandler_price__ew5nF NumberHandler__price">1٫18</div><span title="هزار میلیارد" class="NumberHandler_symbol__2JDUn NumberHandler__symbol">T</span></div></div></td><td class="ant-table-cell"><div style="min-width: 50px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">9٬393</div></div></div></td><td class="ant-table-cell"><div style="min-width: 60px;"><div class="NumberHandler_numberHandler__oHINz"><div class="NumberHandler_price__ew5nF NumberHandler__price">3٬303</div></div></div></td></tr></tbody></table>'

In [8]:
from bs4 import BeautifulSoup
import pandas as pd

# html = """[your HTML table here]"""  # Paste the full HTML table

soup = BeautifulSoup(html, 'html.parser')

# Extract headers
headers = [th.get_text(strip=True) for th in soup.select('thead th')]

# Extract rows
data = []
for row in soup.select('tbody tr'):
    row_data = []
    for td in row.select('td'):
        # Get the numeric value from complex cells
        num_handler = td.select_one('.NumberHandler_price__ew5nF')
        if num_handler:
            value = num_handler.get_text(strip=True)
            # Handle special cases like "—"
            if 'MutedValue_muted__eLxQJ' in str(td):
                value = ''
        else:
            value = td.get_text(strip=True)
        row_data.append(value)
    data.append(row_data)

# Create DataFrame
df = pd.DataFrame(data, columns=headers)

# Save to CSV
df.to_csv('table_data.csv', index=False, encoding='utf-8-sig')

# Or convert to JSON
json_data = df.to_json(orient='records', force_ascii=False)
# print(json_data)